## Introduction:
According to a new report released by Nielsen Music, on average, Americans now spend just slightly more than 32 hours a week listening to music. This is a staggering 36% increase in 2 years and has been made possible due to the popularity of music streaming sites like Spotify, Pandora, Apple Music etc.    
With such a tremendous growth in the music industry, it becomes crucial to deliver personalized music recommendation to the listeners. This piqued our curiosity to understand the process that goes behind the music recommendation engine and led us to work on this project.  

lastfm is one of the oldest music streaming companies that started providing personalized recommendations to its listeners. Their website is a standing example of the amount of analytics that they are using on their data to extract insights. So we thought of doing the same with the open dataset that we have for lastfm users. Although this dataset is 10 years old, the algorithms and the methologies applied would still be relevant for the latest songs and albums.

### What do we want to do?
To build a recommendation engine that provides personalized recommendations to listeners based on their listening history and visualize the same using advanced visualization tools  

### Data:

    The data is formatted one entry per line as follows (tab separated):

    usersha1-artmbid-artname-plays.tsv:
      user-mboxsha1 \t musicbrainz-artist-id \t artist-name \t plays

    usersha1-profile.tsv:
      user-mboxsha1 \t gender ('m'|'f'|empty) \t age (int|empty) \t country (str|empty) \t signup (date|empty)

Example:

    usersha1-artmbid-artname-plays.tsv:
      000063d3fe1cf2ba248b9e3c3f0334845a27a6bf    af8e4cc5-ef54-458d-a194-7b210acf638f    cannibal corpse    48
      000063d3fe1cf2ba248b9e3c3f0334845a27a6bf    eaaee2c2-0851-43a2-84c8-0198135bc3a8    elis    31
      ...

    usersha1-profile.tsv
      000063d3fe1cf2ba248b9e3c3f0334845a27a6bf    m    19    Mexico    Apr 28, 2008
      
For further details, please refer to the readme file in the folder

**Implicit vs Explicit data** 

Also, the data that we have is implicit data.  
To elaborate, Explicit data is direct preference data from the customers like ratings, likes etc and is often used in collaborative recommendation systems whereas implicit data is the non direct preference data like number of views of a customer, number of times a customer listened to a song or the number of times a customer purchased a particular type of product. In general, we have more noise in implicit data and it takes more effort to make relevant recommendations.  

### How does the data look like?

In [15]:
# Importing all the required libraries
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Load the data
raw_data = pd.read_table('usersha1-artmbid-artname-plays.tsv')
raw_data = raw_data.drop(raw_data.columns[1], axis=1)
raw_data.columns = ['user', 'artist', 'plays']


In [74]:
# Subsetting the data for our analysis
raw_data1 = raw_data[0:2000000]

In [75]:
print('Total number of users in the data:',len(raw_data1['user'].unique()))
print('Total number of artists in the data:',len(raw_data1['artist'].unique()))

Total number of users in the data: 40913
Total number of artists in the data: 110821


There are about 204 nulls in the artist column. Lets drop the null rows from the dataset

In [76]:
# Drop NaN columns
data = raw_data1.dropna()
data = data.copy()

Key links for the below code:  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix  

**Advantages of the CSR format:**
* efficient arithmetic operations CSR + CSR, CSR * CSR, etc.
* efficient row slicing
* fast matrix vector products

### How to deal with implicit data?
(You can skip this part if you want to look at the code)

As we all know that there are 2 main recommendation system algorithms
1. Content based approach
2. Collaborative filtering approach  

Collaborative filtering is an approach based on past user behavior without requiring the creation of explicit profiles like in content based approach. Historically, recommendation systems depended on more information rich explicit datasets like ratings and thumbs up/down indications. However, explicit feedback is not always avaialable and it becomes crucial to make use of the vast amount of implicit data like user transaction data, browsing data or user play data which indirectly reflect opinion through observing user behavior.   
Also, it is important to note that once the user gives permission to access the usage data, there is no need to collect explicit data as we can use implicit data to make recommendations.  

**Coming to our dataset**  
There are some unique characteristics of implicit data that prevents the use of conventional algorithms that were successful for explicit data.Lets quickly look into those characteristics briefly.  
1. No negative feedback:
    In explicit dataset, the user explicitly gives the information about his/her likes or dislikes and any other user-item pairs that did not have the information are considered are missing data. But in case of implicit, we don't know for sure if the user did not like the movie or did not even know about the movie. Just conisdering the data with positive feedback will not capture the entire picture and will lead to incorrect conclusions. Hence there is a need to address missing data.  
    
2. Implicit data is noisy:
    User might be passively watching a movie or listening to a song and this will constitute to noise as long as the model is concerned.  
    
3. Frequency might not always represent the true user opinion or vice versa  

**Neightborhood models vs Latent factor models:**  
user oriented approach or item-oriented approach in collaborative filtering are the traditional neighborhood models as they consider similarities between users or items while making a recommendation. Item-oriented approach became popular as they performed better than user-oriented approach in terms of accuracy and are easy interpretable as it is easy to say that 2 items are similar than to talk about 2 similar minded individuals.  

Calculating similarties between items would be a realitively simple task when it comes to ratings data as we can use Pearson cofficient to do that. But when it comes to implicit datasets, it becomes a bit complicated as the scale of the frequencies of the metrics are different for different users and might mean different.

This takes us to the latent factor models in which we use matrix factorization methods to uncover the hidden feature vectors for users and items. Refer to the original paper to gain a complete understanding of how this algorithm is being used for implict data. http://yifanhu.net/PUB/cf.pdf

**Matrix factorization**  
The main idea behind matrix factorization is that we estimate user matrix and item matrix from the original sparse matrix by minimizing the cost function that involces 2 important metrics  
Preference : Binary metric that indicates if the customer has listened to the music at least  
Confidence: Metric that indicates if the user likes the song  

**Alternating least squares**  
It becomes computationally expensive to optimize the cost function with stochastic gradient as the total combinations of users and items will easily reach billions in case of real world datasets. Hence, an alternative approach called Alternating least squares to perform the optimization of the cost function. In this approach, the optimal estimates for the user vector and item vectors are estimated in an alternating manner by keeping one matrix constant and finally arrives at the optimal solution. This is very clearly stated in the paper that was mentioned. For example, in a particular iteration, we will have user vector constant while optimizing for item vector.  

This should give you enough information about the theory behind alternating least squares for implicit dataset if you have read the brief and the paper.  

**Shout out for Ben frederickson**  
Thanks to Ben Frederickson, the entire algorithm is available as a module called 'Implicit' in python and provides a much cleaner approach to get recommendations instead of writing it from scratch and the execution speed has been enhanced by implementing it in Cython. We have used the same while building this recommendation engine.  



### Prerequisites 
If you are running this on your personal system, please make sure that you perform the following steps:  
Before starting the analysis, make sure to download the implicit package  
Run the following command on your command prompt  
*conda install -c conda-forge implicit*

### Lets move on to the execution part of the algorithm

In [77]:
# Converting the numbers to categories to be used for creating the categorical codes to avoid using long hash keys 
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")

#cat.codes creates a categorical id for the users and artists
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matrices, one for fitting the model (item-user) 
# and one for recommendations (user-item)

sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

In [78]:
matrix_size = sparse_user_item.shape[0]*sparse_user_item.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_user_item.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.95588936009682

We have very high sparsity in our data. This might not result in favorable results at the end. But one thing we can be sure of if finding similar artists using the above data. As we will be having enough for each artist based on the people that have listened to them. Recommendations to the users might be less than expected

### Creating train and test data

In [223]:
import random

def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of item,user index into list

    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of item-user pairs without replacement

    item_inds = [index[0] for index in samples] # Get the item row indices

    user_inds = [index[1] for index in samples] # Get the user column indices

    
    training_set[item_inds, user_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user columns that were altered

Lets use the above function to create the training and test sets for our analysis

In [224]:
# 20% of the data has been masked for this exercise
product_train, product_test, product_users_altered = make_train(sparse_item_user, pct_test = 0.05)

### Building the recommendation system

Here we will use the implicit package from python to build the recommendation engine.

In [90]:
# Initialize the als model and fit it using the sparse item-user matrix
# Parameters that we have chosen
# 1. factors = 20 -- Latent factors for user and item vectors
# 2. iterations = 20 -- Number of iterations to use while fitting the data
# 3. regularization = 0.1 -- regularization constant to be used in the cost function

model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=40)

# Calculate the confidence by multiplying it by our alpha value.(alpha value corresponds to the confidence metric 
# that we discussed earlier)

alpha_val = 15
data_conf = (product_train * alpha_val).astype('double')

# We have used an alpha_val of 15 after performing some iterations with different alpha values
#Fit the model
model.fit(data_conf)

100%|████████████████████████████████████████████████████████████████████████████████| 40.0/40 [00:10<00:00,  3.64it/s]


### Bayesian personalized ranking

In this step ,we will be creating extracting the item_vector matrix and user_vector matrix that we have created through the model

In [91]:
item_vecs = model.item_factors
user_vecs = model.user_factors

In [98]:
print('Shape of Artist vector matrix : ', item_vecs.shape)
print('Shape of User vector matrix : ', user_vecs.shape)

Shape of Artist vector matrix :  (110820, 20)
Shape of User vector matrix :  (40913, 20)


### Evaluating the recommendation system using AUC-ROC curve
The function below will help in evaluating our recommendation system by calculating the AUC - ROC curve

In [184]:
from sklearn import metrics
import matplotlib.pylab as plt
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

In [185]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_column = training_set[:,user].toarray().reshape(-1) # Get the training set column
        zero_inds = np.where(training_column == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,user].toarray()[zero_inds,0].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [186]:
calc_mean_auc(product_train, product_users_altered,
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.962, 0.934)

We are actually getting a better AUC in terms of prediction of artists for a user. But there are a couple of points that we need to consider there.
1. By calculating AUC, we are actually not considering the rank of the recommendation. Although, we might be predicting that the artist, the artist the user has listened might not be present in the top 20 recommendations.  
2. Because of the sparsity of the data, we can see that there will be a lot more True negatives than false positives effectively lowering the False positive rate. This also contributes to the increase in AUC.  

Having said that, let's see how our model performs when it comes to identifying similar artists and making recommendations to the users

### Example for recommendation

### Finding similar artists

In [188]:
# Lets start with red hot chilli peppers. Earlier in our exploration, we have seen that artist_id for red hot chilli peppers
# is 220128
data[data['artist'] == 'red hot chili peppers'].head(5)

,user,artist,plays,user_id,artist_id
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691,0,80876
1422,000429493d9716b66b02180d208d09b5b89fbe64,red hot chili peppers,234,29,80876
2139,0007e26aafcfc0b6dcb87d7041583fbb7cced88a,red hot chili peppers,159,44,80876
3284,000b0bb32f149504e1df3cce85b6bfd20cef3dd0,red hot chili peppers,46,68,80876
3322,000b2ee840cbda56e0f41c8f248c4fb7ee275db3,red hot chili peppers,87,69,80876


In [191]:
# Find the 10 most similar to red hot chilli peppers
artist_id = 80876
n_similar = 10 # getting the top ten similar items

# Use implicit to get similar items.
similar = model.similar_items(artist_id, n_similar)
# Print the names of our most similar artists
for artist in similar:
    idx, score = artist
    print (data.artist.loc[data.artist_id == idx].iloc[0])

C:\Users\Akhil\Anaconda2\envs\py36\lib\site-packages\numpy\linalg\linalg.py:2390: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


red hot chili peppers
muse
nirvana
placebo
coldplay
the beatles
pink floyd
foo fighters
the killers
nine inch nails


In a similar way, lets look atsome other band

In [195]:
data[data['artist'] == 'die Ärzte'].head(5)

,user,artist,plays,user_id,artist_id
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,0,30264
2943,000a1585c5f65532a9c9187a882892982d345a5c,die Ärzte,148,61,30264
3787,000cb6427411006fe9a6193d3c4f59efed53fbef,die Ärzte,7,78,30264
6295,0014ffc91d3a5b59cce9bceaf22ef0d72e5711b8,die Ärzte,88,128,30264
13513,003059a886782e4d7936da913d3f064f637d0b2b,die Ärzte,5,274,30264


In [196]:
# Find the 10 most similar to red hot chilli peppers
artist_id = 30264
n_similar = 10 # getting the top ten similar items

# Use implicit to get similar items.
similar = model.similar_items(artist_id, n_similar)
# Print the names of our most similar artists
for artist in similar:
    idx, score = artist
    print (data.artist.loc[data.artist_id == idx].iloc[0])

die Ärzte
ska-p
guano apes
no doubt
3 doors down
mando diao
amy macdonald
nickelback
bon jovi
bloodhound gang


#### Insights:
* Also, one more example that we took was die Ärzte(90933). It is a punk rock from Berlin according to wiki   https://en.wikipedia.org/wiki/Die_%C3%84rzte  
* The top recommendations for their music are die toten hosen(another german punk rock), blink-182(punk rock).  


**It's really amazing how math works. Without mentioning any other features, the algorithm figured out the features vectors for each artist based on the number of times the users played their songs.**  

### Creating user recommendations

Lets look at the users who have rock music on the top of their list by doing a simple EDA and find the recommendations for those users using the algorithm

In [198]:
data['rank'] = data.groupby(['user_id'])['plays'].rank(ascending = False)

# filtering for their first choice
data_1  = data[data['rank'] == 1]

In [211]:
# Users with red hot chilli peppers as their first choice
data_1[data_1['artist_id'] == 30264].head()

,user,artist,plays,user_id,artist_id,rank
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099,0,30264,1.0
25686,005d521c9f8b1acfc13b7a4cc4b39085edfc786a,die Ärzte,933,523,30264,1.0
29112,006aaaaf386fdbb0aea3f2bf9019e346a7294b6a,die Ärzte,946,595,30264,1.0
30440,006f93b4213be13020a3819e0d0a86dcf97b58de,die Ärzte,1375,622,30264,1.0
53937,00c465e5b33365ab91cc5cf161590a38044954af,die Ärzte,2924,1094,30264,1.0


In [217]:
data[data['user_id'] == 1094].head(10)

,user,artist,plays,user_id,artist_id,rank
53937,00c465e5b33365ab91cc5cf161590a38044954af,die Ärzte,2924,1094,30264,1.0
53938,00c465e5b33365ab91cc5cf161590a38044954af,equilibrium,1936,1094,36491,2.0
53939,00c465e5b33365ab91cc5cf161590a38044954af,ensiferum,1782,1094,36343,3.0
53940,00c465e5b33365ab91cc5cf161590a38044954af,system of a down,1167,1094,92524,4.0
53941,00c465e5b33365ab91cc5cf161590a38044954af,sdp,1042,1094,85406,5.0
53942,00c465e5b33365ab91cc5cf161590a38044954af,deichkind,1013,1094,29016,6.0
53943,00c465e5b33365ab91cc5cf161590a38044954af,knorkator,920,1094,57157,7.0
53944,00c465e5b33365ab91cc5cf161590a38044954af,typ:t.u.r.b.o.,911,1094,103029,8.0
53945,00c465e5b33365ab91cc5cf161590a38044954af,serj tankian,813,1094,85997,9.0
53946,00c465e5b33365ab91cc5cf161590a38044954af,rise against,779,1094,82040,10.0


In [216]:
# Create recommendations for user with id 1246
user_id = 1094

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item,N = 20,filter_already_liked_items = False)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print (recommendations)

               artist     score
0               lafee  1.328061
1                 ost  1.282044
2      disco ensemble  1.216249
3          eisbrecher  1.200554
4                4lyn  1.199269
5       jan hegenberg  1.195768
6   blood stain child  1.182727
7         eläkeläiset  1.181833
8           krypteria  1.167898
9        freedom call  1.163600
10        celldweller  1.161240
11    sonic syndicate  1.155655
12       revolverheld  1.153287
13        dieter nuhr  1.149378
14          fightstar  1.147582
15      powerman 5000  1.146510
16      stephen lynch  1.144552
17            volbeat  1.143474
18   disarmonia mundi  1.138794
19              blind  1.137219


We can observe, although the model is not exactly recommending there top 10 again, the recommendations are still closer to what they have listened to. In this case, German music and punk bands.  

### Conclusion:  

We can still use the similar artists part of this model to make suggestions to the users based on the artists that they like.This will still provide an improvement experience to the users.  

For the exact personalized recommendations to users, there are some key tasks that we need to perform to improve the result.
1. Find a work around to deal with the sparsity of the data  
2. Bring in the profile data and search data of the users to overlay on top of the recommendations to create an ensembled result  
3. Apply more sophisticated algorithms like Bayesian personalied ranking that takes the ranking into consideration while making recommendations.  
4. Use Factorization machines to deal with the sparsity of the data  
5. Final use deep learning techniques to find similarities between the music in itself and overlay those results on to the collaborative filtering recommendations  

References:
1. https://implicit.readthedocs.io/en/latest/als.html  
2. https://jessesw.com/Rec-System/  